Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [1]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:080d344ba335d2131ae15769dd8daf149acaa5a1f122df998c2c7223231a62f4
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from create-input
Digest: sha256:cb94537e14345c9f0a03014566006658b030c5fed66f35bb511471838c733df0
Status: Image is up to date for ziylregistry.azurecr.io/create-input:latest
Using default tag: latest
latest: Pulling from perf_test
Digest: sha256:0b93e6a1d4e4cd5e0057cf503fce53b8702d2445252b7837e844e50752d2a369
Status: Image is up to date for ziylregistry.azurecr.io/perf_test:latest


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

You can run the docker image with customized parameters.

In [3]:
# docker run -v [Your model path]:/mnt/model.tf ziylregistry.azurecr.io/onnx-converter --model /mnt/model.tf --output_onnx_path [Output path on the docker] --model_type [Model type]
# for windows
#!docker run -v %CD%\model:/mnt/model ziylregistry.azurecr.io/onnx-converter --model /mnt/model --output_onnx_path /mnt/output/converted.onnx --model_type tensorflow
    
# for Unix 
!docker run -v $PWD/model:/mnt/model ziylregistry.azurecr.io/onnx-converter --model /mnt/model --output_onnx_path /mnt/output/converted.onnx --model_type tensorflow

2019-05-29 21:24:05.090888: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
INFO:tensorflow:Restoring parameters from /mnt/model/variables/variables
INFO:tensorflow:Restoring parameters from /mnt/model/variables/variables
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.
INFO:tensorflow:Converted 9 variables to const ops.
2019-05-29 21:24:05.248444: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying fold_batch_norms
2019-05-29 21:24:05.261772: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying fold_old_batch_norms
INFO:tf2onnx.optimizer.transpose_optimizer: 2 transpose op(s) left, ops diff after transpose optimization: Counter({'Const': 1, 'Gather': 1, 'Unsqueeze': 0, 'Placeholder': 0, 'Shape': 0, 'Cast': 0, 'Slice': 0, 'Squeeze': 0, 'Concat': 0, 'Reshape': 0, 'Conv': 0, 'Rel

## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs: string

    Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs: string

    Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string

    Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple

    Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.


# Generate-Input image

This folder contains source code for generate-input image. Generate-input image is used to generate randomized inputs for a specified model if there's no input files exist. After running this image, a ./test_data_set_0 folder will be created with input_*.pb files at the same directory of the input model file.

You can run the docker image with customized parameters.

In [10]:
# docker run -v [Your model path]:/mnt/model.onnx ziylregistry.azurecr.io/create-input --model /mnt/model.onnx
# for Windows
#!docker run -v %CD%\deepthink.onnx:/mnt/model.onnx ziylregistry.azurecr.io/create-input --model /mnt/model.onnx

# for Unix
!docker run -v $PWD/deepthink.onnx:/mnt/output/model.onnx ziylregistry.azurecr.io/create-input --model /mnt/output/model.onnx

/mnt/output/model.onnx inputs: 
input name: long_MRC_2/passage_mask:0, shape: [32, None, 40], type: tensor(float)
input name: long_MRC_2/passage:0, shape: [32, None, 40], type: tensor(int32)
input name: long_MRC_2/target_mask:0, shape: [32], type: tensor(int32)
input name: long_MRC_2/start_only_label:0, shape: [32, None], type: tensor(float)
input name: long_MRC_2/sent_passage_mask:0, shape: [32, None], type: tensor(float)
input name: long_MRC_2/ques_mask:0, shape: [None, 40], type: tensor(float)
input name: long_MRC_2/ques_input:0, shape: [None, 40], type: tensor(int32)
Randomized input .pb file generated at  /mnt/output/test_data_set_0


## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [14]:
# docker run -v [Your model path]:/mnt/model ziylregistry.azurecr.io/perf_test /mnt/model [Output path on the docker]
# for Windows
# docker run -v %CD%\model:/mnt/model ziylregistry.azurecr.io/perf_test /mnt/model /mnt/result
# for Unix
!docker run -v $PWD/20190529:/mnt/model ziylregistry.azurecr.io/perf_test /mnt/model/elmo.onnx /mnt/result

Cores:  6

/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32519 ; hostname=15ff103794b3 ; expr=cudaSetDevice(device_id_); 
Stacktrace:

Stacktrace:



Setting thread pool size to 0


/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [w


Setting thread pool size to 0
Total time cost:26.8981
Total iterations:10
Average time cost:2689.81 ms


2019-05-29 22:00:39.085708127 [W:onnxruntime:Default, graph.cc:2208 CleanUnusedInitializers] bilm/strided_slice_20/stack_1/1:0 exists in this graph's initializers but it is not used by any node
2019-05-29 22:00:39.085747069 [W:onnxruntime:Default, graph.cc:2208 CleanUnusedInitializers] bilm/CNN/W_cnn_5/read/_5__cf__5:0 exists in this graph's initializers but it is not used by any node
2019-05-29 22:00:39.085752200 [W:onnxruntime:Default, graph.cc:2208 CleanUnusedInitializers] bilm/strided_slice_3/stack_1/1:0 exists in this graph's initializers but it is not used by any node
2019-05-29 22:00:39.085756903 [W:onnxruntime:Default, graph.cc:2208 CleanUnusedInitializers] end_mask__237 exists in this graph's initializers but it is not used by any node
2019-05-29 22:00:39.085761584 [W:onnxruntime:Default, graph.cc:2208 CleanUnusedInitializers] bilm/strided_slice_8/stack_1/1:0 exists in thi


Setting thread pool size to 0
Total time cost:26.1813
Total iterations:10
Average time cost:2618.13 ms

ngraph 2618.13

/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32566 ; hostname=15ff103794b3 ; expr=cudaSetDevice(device_id_); 
Stacktrace:

Stacktrace:



Setting thread pool size to 0


/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/provide


mkldnn_openmp OMP_NUM_THREADS=1, active 2504.29


Setting thread pool size to 0
Total time cost:11.5292
Total iterations:10
Average time cost:1152.92 ms



Setting thread pool size to 0
Total time cost:11.4873
Total iterations:10
Average time cost:1148.73 ms

mkldnn_openmp passive 1148.73


Setting thread pool size to 0
Total time cost:8.4307
Total iterations:10
Average time cost:843.07 ms



Setting thread pool size to 0
Total time cost:8.34259
Total iterations:10
Average time cost:834.259 ms

mkldnn_openmp active 834.259


Setting thread pool size to 0
Total time cost:8.15352
Total iterations:10
Average time cost:815.352 ms



Setting thread pool size to 0
Total time cost:8.01235
Total iterations:10
Average time cost:801.235 ms

cpu 801.235


Setting thread pool size to 0
Total time cost:8.35991
Total iterations:10
Average time cost:835.991 ms



Setting thread pool size to 0
Total time cost:8.46642
Total iterations:10
Average time cost:846.642 ms

mkldnn 846.642

Results:
cpu_openm

# Run parameters

(1) convert-to-onnx-output: string

    Required. The path of the model that wants to be performed.
    
(2) output-perf-result-path: string

    Required. The path of the result.
    